### Imports

In [1]:
import torch
import monai

from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd, 
    EnsureTyped,
    EnsureType,
    Invertd
)

# from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import nibabel as nib
import numpy as np
from tqdm.notebook import tqdm

### Dataset

In [2]:
root_dir = "/scratch/scratch6/akansh12/Parse_data/train/train/"
train_images = sorted(glob.glob(os.path.join(root_dir, "*", 'image', "*.nii.gz")))
train_labels = sorted(glob.glob(os.path.join(root_dir, "*", 'label', "*.nii.gz")))

data_dicts = [{"images": images_name, "labels": label_name} for images_name, label_name in zip(train_images, train_labels)]
train_files, val_files = data_dicts[:-9], data_dicts[-9:]
set_determinism(seed = 0)

### Transforms

In [3]:
train_transforms = Compose(
    [
     LoadImaged(keys=['images', 'labels']),
     EnsureChannelFirstd(keys = ["images", "labels"]),
     Orientationd(keys=['images', 'labels'], axcodes = 'LPS'),
     Spacingd(keys=['images', 'labels'], pixdim = (1.5,1.5,2), mode = ("bilinear", 'nearest')),
     ScaleIntensityRanged(
            keys=["images"], a_min=-700, a_max=300,
            b_min=0.0, b_max=1.0, clip=True,
        ),
     CropForegroundd(keys=['images', 'labels'], source_key="images"),
     RandCropByPosNegLabeld(
            keys=['images', 'labels'],
            label_key="labels",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="images",
            image_threshold=0,
        ),
     EnsureTyped(keys=['images', 'labels']),     
          
    ]
)

val_transforms = Compose(
    [
        LoadImaged(keys=["images", "labels"]),
        EnsureChannelFirstd(keys=["images", "labels"]),
        Orientationd(keys=["images", "labels"], axcodes="LPS"),
        Spacingd(keys=["images", "labels"], pixdim=(
            1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
        ScaleIntensityRanged(
            keys=["images"], a_min=-700, a_max=300,
            b_min=0.0, b_max=1.0, clip=True,
        ),
        CropForegroundd(keys=["images", "labels"], source_key="images"),
        EnsureTyped(keys=["images", "labels"]),
    ]
)

### DataLoader

In [4]:
train_ds = CacheDataset(
    data = train_files, transform = train_transforms,
    cache_rate = 1.0, num_workers = 4
)

train_loader = DataLoader(train_ds, batch_size = 2, shuffle = True, num_workers=4)
val_ds = CacheDataset(
    data = val_files, transform = val_transforms,
    cache_rate = 1.0, num_workers = 4
)
val_loader = DataLoader(val_ds, batch_size = 1, shuffle = False, num_workers=4)

Loading dataset: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:33<00:00,  3.77s/it]


### Model defination

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") 

UNet_meatdata = dict(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH
)
model = UNet(**UNet_meatdata).to(device)

### Loss function and Optimizers

In [6]:
loss_function = DiceLoss(to_onehot_y=True, softmax=True)
loss_type = "DiceLoss"
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
dice_metric = DiceMetric(include_background=False, reduction="mean")

In [7]:
Optimizer_metadata = {}
for ind, param_group in enumerate(optimizer.param_groups):
    optim_meta_keys = list(param_group.keys())
    Optimizer_metadata[f'param_group_{ind}'] = {key: value for (key, value) in param_group.items() if 'params' not in key}

In [ ]:
max_epochs = 600
val_interval = 10
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
post_pred = Compose([EnsureType(), AsDiscrete(argmax=True, to_onehot=2)])
post_label = Compose([EnsureType(), AsDiscrete(to_onehot=2)])


slice_to_track = 80

for epoch in tqdm(range(max_epochs)):
    model.train()
    epoch_loss = 0
    step = 0
    
    for batch_data in train_loader:
        step += 1
        inputs, labels = (
            batch_data['images'].to(device),
            batch_data['labels'].to(device)
        )
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss +=loss.item()
        print(f"{step}/{len(train_ds) // train_loader.batch_size}, "
                f"train_loss: {loss.item():.4f}")

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)

    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:

        model.eval()
        with torch.no_grad():
            for index, val_data in enumerate(val_loader):

                val_inputs, val_labels = val_data['images'].to(device), val_data['labels'].to(device)
                roi_size = (160, 160, 160)
                sw_batch_size = 4
                val_outputs = sliding_window_inference(
                            val_inputs, roi_size, sw_batch_size, model)

                output = torch.argmax(val_outputs, dim=1)[0, :, :, slice_to_track].float()

                val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
                val_labels = [post_label(i) for i in decollate_batch(val_labels)]
                dice_metric(y_pred=val_outputs, y=val_labels)

                metric = dice_metric.aggregate().item()
                dice_metric.reset()

            metric_values.append(metric)
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), os.path.join(
                    root_dir, "/scratch/scratch6/akansh12/challenges/parse2022/temp/best_metric_model.pth"))

                best_model_log_message = f"saved new best metric model at the {epoch+1}th epoch"
                print(best_model_log_message)

                message1 = f"current epoch: {epoch + 1} current mean dice: {metric:.4f}"
                message2 = f"\nbest mean dice: {best_metric:.4f} "
                message3 = f"at epoch: {best_metric_epoch}"
                print(message1, message2, message3)

np.save("/scratch/scratch6/akansh12/challenges/parse2022/temp/epoch_loss.npy", epoch_loss_values)
np.save("/scratch/scratch6/akansh12/challenges/parse2022/temp/metric_values.npy", metric_values)

  0%|          | 0/600 [00:00<?, ?it/s]

1/45, train_loss: 0.6929
2/45, train_loss: 0.7021
3/45, train_loss: 0.6923
4/45, train_loss: 0.6900
5/45, train_loss: 0.6879
6/45, train_loss: 0.6898
7/45, train_loss: 0.6857
8/45, train_loss: 0.6875
9/45, train_loss: 0.6886
10/45, train_loss: 0.6846
11/45, train_loss: 0.6766
12/45, train_loss: 0.6795
13/45, train_loss: 0.6802
14/45, train_loss: 0.6713
15/45, train_loss: 0.6735
16/45, train_loss: 0.6761
17/45, train_loss: 0.6770
18/45, train_loss: 0.6730
19/45, train_loss: 0.6725
20/45, train_loss: 0.6667
21/45, train_loss: 0.6710
22/45, train_loss: 0.6723
23/45, train_loss: 0.6688
24/45, train_loss: 0.6673
25/45, train_loss: 0.6665
26/45, train_loss: 0.6656
27/45, train_loss: 0.6629
28/45, train_loss: 0.6638
29/45, train_loss: 0.6608
30/45, train_loss: 0.6640
31/45, train_loss: 0.6593
32/45, train_loss: 0.6531
33/45, train_loss: 0.6605
34/45, train_loss: 0.6619
35/45, train_loss: 0.6587
36/45, train_loss: 0.6519
37/45, train_loss: 0.6532
38/45, train_loss: 0.6611
39/45, train_loss: 0.

36/45, train_loss: 0.5555
37/45, train_loss: 0.5414
38/45, train_loss: 0.5458
39/45, train_loss: 0.5489
40/45, train_loss: 0.5581
41/45, train_loss: 0.5577
42/45, train_loss: 0.5643
43/45, train_loss: 0.5540
44/45, train_loss: 0.5560
45/45, train_loss: 0.5567
46/45, train_loss: 0.5319
epoch 7 average loss: 0.5556
1/45, train_loss: 0.5517
2/45, train_loss: 0.5464
3/45, train_loss: 0.5401
4/45, train_loss: 0.5628
5/45, train_loss: 0.5463
6/45, train_loss: 0.5481
7/45, train_loss: 0.5537
8/45, train_loss: 0.5414
9/45, train_loss: 0.5614
10/45, train_loss: 0.5435
11/45, train_loss: 0.5185
12/45, train_loss: 0.5319
13/45, train_loss: 0.5525
14/45, train_loss: 0.5371
15/45, train_loss: 0.5375
16/45, train_loss: 0.5329
17/45, train_loss: 0.5422
18/45, train_loss: 0.5484
19/45, train_loss: 0.5369
20/45, train_loss: 0.5492
21/45, train_loss: 0.5527
22/45, train_loss: 0.5443
23/45, train_loss: 0.5392
24/45, train_loss: 0.5440
25/45, train_loss: 0.5516
26/45, train_loss: 0.5549
27/45, train_loss:

19/45, train_loss: 0.4461
20/45, train_loss: 0.4773
21/45, train_loss: 0.4704
22/45, train_loss: 0.4935
23/45, train_loss: 0.4597
24/45, train_loss: 0.4757
25/45, train_loss: 0.4654
26/45, train_loss: 0.4758
27/45, train_loss: 0.4825
28/45, train_loss: 0.4206
29/45, train_loss: 0.4744
30/45, train_loss: 0.4593
31/45, train_loss: 0.4723
32/45, train_loss: 0.4870
33/45, train_loss: 0.4761
34/45, train_loss: 0.4917
35/45, train_loss: 0.4710
36/45, train_loss: 0.4646
37/45, train_loss: 0.5128
38/45, train_loss: 0.4792
39/45, train_loss: 0.4558
40/45, train_loss: 0.4480
41/45, train_loss: 0.4932
42/45, train_loss: 0.4612
43/45, train_loss: 0.4504
44/45, train_loss: 0.4882
45/45, train_loss: 0.4442
46/45, train_loss: 0.4427
epoch 14 average loss: 0.4708
1/45, train_loss: 0.4484
2/45, train_loss: 0.4461
3/45, train_loss: 0.4619
4/45, train_loss: 0.4771
5/45, train_loss: 0.4486
6/45, train_loss: 0.4909
7/45, train_loss: 0.4829
8/45, train_loss: 0.4327
9/45, train_loss: 0.4701
10/45, train_loss

7/45, train_loss: 0.4161
8/45, train_loss: 0.3729
9/45, train_loss: 0.4000
10/45, train_loss: 0.3713
11/45, train_loss: 0.3471
12/45, train_loss: 0.3615
13/45, train_loss: 0.3748
14/45, train_loss: 0.3608
15/45, train_loss: 0.4280
16/45, train_loss: 0.3389
17/45, train_loss: 0.4037
18/45, train_loss: 0.4116
19/45, train_loss: 0.3437
20/45, train_loss: 0.3660
21/45, train_loss: 0.3566
22/45, train_loss: 0.3581
23/45, train_loss: 0.3811
24/45, train_loss: 0.3882
25/45, train_loss: 0.3853
26/45, train_loss: 0.3679
27/45, train_loss: 0.3428
28/45, train_loss: 0.3430
29/45, train_loss: 0.3647
30/45, train_loss: 0.3941
31/45, train_loss: 0.4244
32/45, train_loss: 0.4052
33/45, train_loss: 0.3591
34/45, train_loss: 0.3379
35/45, train_loss: 0.3633
36/45, train_loss: 0.3691
37/45, train_loss: 0.3737
38/45, train_loss: 0.4290
39/45, train_loss: 0.4027
40/45, train_loss: 0.3581
41/45, train_loss: 0.3761
42/45, train_loss: 0.3877
43/45, train_loss: 0.3864
44/45, train_loss: 0.3702
45/45, train_lo

...

24/45, train_loss: 0.0938
25/45, train_loss: 0.0775
26/45, train_loss: 0.0730
27/45, train_loss: 0.0617
28/45, train_loss: 0.0731
29/45, train_loss: 0.0875
30/45, train_loss: 0.0641
31/45, train_loss: 0.0514
32/45, train_loss: 0.0701
33/45, train_loss: 0.0435
34/45, train_loss: 0.0606
35/45, train_loss: 0.0492
36/45, train_loss: 0.0887
37/45, train_loss: 0.0479
38/45, train_loss: 0.0747
39/45, train_loss: 0.0659
40/45, train_loss: 0.0737
41/45, train_loss: 0.0504
42/45, train_loss: 0.0629
43/45, train_loss: 0.0637
44/45, train_loss: 0.0667
45/45, train_loss: 0.0628
46/45, train_loss: 0.0790
epoch 457 average loss: 0.0709
1/45, train_loss: 0.0784
2/45, train_loss: 0.0772
3/45, train_loss: 0.0649
4/45, train_loss: 0.0515
5/45, train_loss: 0.0631
6/45, train_loss: 0.0819
7/45, train_loss: 0.0472
8/45, train_loss: 0.0760
9/45, train_loss: 0.0668
10/45, train_loss: 0.0542
11/45, train_loss: 0.0647
12/45, train_loss: 0.0657
13/45, train_loss: 0.0946
14/45, train_loss: 0.0677
15/45, train_los

12/45, train_loss: 0.0676
13/45, train_loss: 0.0620
14/45, train_loss: 0.0744
15/45, train_loss: 0.0550
16/45, train_loss: 0.0488
17/45, train_loss: 0.0994
18/45, train_loss: 0.0736
19/45, train_loss: 0.0674
20/45, train_loss: 0.0575
21/45, train_loss: 0.0858
22/45, train_loss: 0.0680
23/45, train_loss: 0.0817
24/45, train_loss: 0.0543
25/45, train_loss: 0.0750
26/45, train_loss: 0.0643
27/45, train_loss: 0.0872
28/45, train_loss: 0.0590
29/45, train_loss: 0.0741
30/45, train_loss: 0.0685
31/45, train_loss: 0.0623
32/45, train_loss: 0.0888
33/45, train_loss: 0.0716
34/45, train_loss: 0.0779
35/45, train_loss: 0.0647
36/45, train_loss: 0.0671
37/45, train_loss: 0.0627
38/45, train_loss: 0.0591
39/45, train_loss: 0.1175
40/45, train_loss: 0.0544
41/45, train_loss: 0.0774
42/45, train_loss: 0.0686
43/45, train_loss: 0.0647
44/45, train_loss: 0.0672
45/45, train_loss: 0.0602
46/45, train_loss: 0.0735
epoch 464 average loss: 0.0714
1/45, train_loss: 0.0475
2/45, train_loss: 0.0605
3/45, tra

1/45, train_loss: 0.1211
2/45, train_loss: 0.0813
3/45, train_loss: 0.0568
4/45, train_loss: 0.0940
5/45, train_loss: 0.0862
6/45, train_loss: 0.0586
7/45, train_loss: 0.0692
8/45, train_loss: 0.0714
9/45, train_loss: 0.0706
10/45, train_loss: 0.1021
11/45, train_loss: 0.0894
12/45, train_loss: 0.0583
13/45, train_loss: 0.0691
14/45, train_loss: 0.0760
15/45, train_loss: 0.0819
16/45, train_loss: 0.0866
17/45, train_loss: 0.0609
18/45, train_loss: 0.0715
19/45, train_loss: 0.0663
20/45, train_loss: 0.0718
21/45, train_loss: 0.0579
22/45, train_loss: 0.0787
23/45, train_loss: 0.0901
24/45, train_loss: 0.0693
25/45, train_loss: 0.0973
26/45, train_loss: 0.0596
27/45, train_loss: 0.0684
28/45, train_loss: 0.0774
29/45, train_loss: 0.0480
30/45, train_loss: 0.0651
31/45, train_loss: 0.0851
32/45, train_loss: 0.0747
33/45, train_loss: 0.0692
34/45, train_loss: 0.0595
35/45, train_loss: 0.0830
36/45, train_loss: 0.0835
37/45, train_loss: 0.0692
38/45, train_loss: 0.0615
39/45, train_loss: 0.

36/45, train_loss: 0.0669
37/45, train_loss: 0.0574
38/45, train_loss: 0.0726
39/45, train_loss: 0.0740
40/45, train_loss: 0.0637
41/45, train_loss: 0.0910
42/45, train_loss: 0.0565
43/45, train_loss: 0.0617
44/45, train_loss: 0.0649
45/45, train_loss: 0.0764
46/45, train_loss: 0.0630
epoch 477 average loss: 0.0695
1/45, train_loss: 0.0620
2/45, train_loss: 0.0805
3/45, train_loss: 0.0721
4/45, train_loss: 0.0666
5/45, train_loss: 0.0718
6/45, train_loss: 0.0488
7/45, train_loss: 0.0585
8/45, train_loss: 0.0864
9/45, train_loss: 0.0842
10/45, train_loss: 0.1233
11/45, train_loss: 0.0735
12/45, train_loss: 0.0846
13/45, train_loss: 0.1462
14/45, train_loss: 0.0775
15/45, train_loss: 0.0663
16/45, train_loss: 0.0539
17/45, train_loss: 0.0782
18/45, train_loss: 0.0616
19/45, train_loss: 0.0520
20/45, train_loss: 0.0646
21/45, train_loss: 0.0755
22/45, train_loss: 0.0714
23/45, train_loss: 0.0571
24/45, train_loss: 0.0891
25/45, train_loss: 0.1350
26/45, train_loss: 0.0885
27/45, train_los

24/45, train_loss: 0.0636
25/45, train_loss: 0.0639
26/45, train_loss: 0.0619
27/45, train_loss: 0.0730
28/45, train_loss: 0.0666
29/45, train_loss: 0.0758
30/45, train_loss: 0.0752
31/45, train_loss: 0.0828
32/45, train_loss: 0.0483
33/45, train_loss: 0.0776
34/45, train_loss: 0.0618
35/45, train_loss: 0.1112
36/45, train_loss: 0.0788
37/45, train_loss: 0.0683
38/45, train_loss: 0.0650
39/45, train_loss: 0.0988
40/45, train_loss: 0.0754
41/45, train_loss: 0.0535
42/45, train_loss: 0.0725
43/45, train_loss: 0.0676
44/45, train_loss: 0.0657
45/45, train_loss: 0.0566
46/45, train_loss: 0.0752
epoch 484 average loss: 0.0712
1/45, train_loss: 0.0597
2/45, train_loss: 0.1147
3/45, train_loss: 0.0774
4/45, train_loss: 0.0784
5/45, train_loss: 0.0740
6/45, train_loss: 0.0717
7/45, train_loss: 0.0512
8/45, train_loss: 0.0725
9/45, train_loss: 0.0517
10/45, train_loss: 0.0760
11/45, train_loss: 0.0771
12/45, train_loss: 0.0592
13/45, train_loss: 0.0541
14/45, train_loss: 0.0509
15/45, train_los